Trap Depth Calculations

In [4]:
import numpy as np
import scipy.constants as sc

In [5]:
def gamma_w_light(omega: float) -> float:
    # Equation (7) of Grimm, unit = Hz (or rads/s)
    # Scattering
    result = (sc.elementary_charge**2) * (omega**2)
    result /= 6 * sc.pi * sc.epsilon_0 * sc.electron_mass * (sc.speed_of_light**3)

    return result

In [6]:
def to_freq_ang(wavelength: float) -> float:
    # Converts a wavelength in nanometers of light to radians
    return (2*np.pi*sc.speed_of_light*1e9)/(wavelength)

In [10]:
gamma_w_light(omega = to_freq_ang(wavelength = 670.992421))/1e6

49.38394078090274

In [4]:
def alpha(wavelength: float) -> complex:
    # Units = (A^2s^4)/(kg rad)
    omega_0 = to_freq_ang(wavelength = 671)
    omega   = to_freq_ang(wavelength = wavelength)

    result  = sc.elementary_charge**2 / sc.electron_mass
    result /= complex(omega_0**2 - omega**2, - omega*gamma_w_light(omega = omega))

    return result


In [5]:
def trap_depth(intensity: float, wavelength: float) -> float:
    # Equation (2) of Grimm
    _alpha = alpha(wavelength = wavelength)
    print("alpha =", _alpha)

    U_dip  = - np.real(_alpha) * intensity
    U_dip /= 2 * sc.epsilon_0 * sc.speed_of_light

    return U_dip

In [19]:
def trap_temperature(trap_depth: float) -> float:
    return -trap_depth/sc.Boltzmann

In [7]:
from typing import Tuple

def gaussian_beam_width(z: float, w_0: float , z_0: float, Msq: float, wavelength: float) -> float:
    """ Returns the gaussian beam width based on a gaussian beam propagation
    
    Note that this function is normalized if:
	- Everything is in SI-Units, or
	- w, w_0: [um], z, z_0: [mm], lmbda: [nm] (preferred)


    Args:
        z (float): Position in propagation direction [m, mm]
        w_0 (float): Beam waist [m, um]
        z_0 (float): Position of beam waist [m, mm]
        Msq (float): M-squared beam quality factor
        wavelength (float): Wavelength of light [m, nm]

    Returns:
        float: _description_
    """

    return w_0 * np.sqrt(
		1 + ((z - z_0)**2)*((
			(Msq * wavelength)/
			(np.pi * (w_0**2))
		)**2)
	)

def max_intensity(power: float, width_x: float, width_y: float) -> float:
    """Generate the max-intensity of the gaussian intensity distribution given power, w_x and w_y

    Function is normalized if:
    - Everything in SI-units, or
    - P [W], w_x [um], w_y [um] (preferred)

    Args:
        power (float): Power [W]
        width_x (float): width [m, um]
        width_y (float): width [m, um]

    Returns:
        float: Max intensity I_0 in [W/m^2, W/um^2]
    """
    return (2 * power) / (np.pi * (width_x*width_y))

def intensity(
        x: float, y: float, z: float,
        power: float, 
        wavelength: float,
        w_0: Tuple[float, float],
        z_0: Tuple[float, float],
        Msq: Tuple[float, float]
    ) -> float:

    """Returns the intensity of a guassian beam at a point in 3D space. Assumes a simple astigmatic beam

    Note that this function is normalized if:
	- Everything is in SI-Units, or
	- w, w_0: [um], z, z_0: [mm], lmbda: [nm] (preferred)

    Args:
        x (float): x position (one of the main axes)                        [m, um]
        y (float): y position (one of the main axes)                        [m, um]
        z (float): z position (propagation direction)                       [m, mm]
        power (float): Power of the beam                                    [W]
        wavelength (float): Wavelength of the light                         [m, nm]
        w_0 (Tuple[float, float]): Tuple of the beam waist (x, y axes)      [m, um]
        z_0 (Tuple[float, float]): Tuple of the rayleigh length (x, y axes) [m, mm]
        Msq (Tuple[float, float]): Tuple of the beam quality factor M^2 (x, y axes)

    Returns:
        float: The intensity at that point [W/m^2, W/um^2]
    """ 

    # the x and y are the main axes for astigmatism

    x_params = { "w_0": w_0[0], "z_0": z_0[0], "Msq": Msq[0], "wavelength": wavelength }
    y_params = { "w_0": w_0[1], "z_0": z_0[1], "Msq": Msq[1], "wavelength": wavelength }

    w_x = gaussian_beam_width(z = z, **x_params)
    w_y = gaussian_beam_width(z = z, **y_params)

    I0 = max_intensity(power = power, width_x = w_x, width_y = w_y)

    intensity = I0 * np.exp(-2*((x/w_x)**2 + (y/w_y)**2))

    return intensity

# Check units

In [10]:
x = gaussian_beam_width(z = 3,    w_0 = 5, z_0 = 6, Msq = 1, wavelength = 1070)
y = gaussian_beam_width(z = 3e-3, w_0 = 5e-6, z_0 = 6e-3, Msq = 1, wavelength = 1070e-9)

print(x, y * 1e6) # convert to um

204.4161058594956 204.41610585949562


In [24]:
x = max_intensity(power = 100, width_x = 40, width_y = 40)
y = max_intensity(power = 100, width_x = 40e-6, width_y = 40e-6)
print(x, y * 1e-12) # convert to W/um^2

0.039788735772973836 0.03978873577297383


# Calculations

In [26]:
U = trap_depth(x*1e12, 1070) # SI UNIT OUTPUT
temp = trap_temperature(U)

print(U, temp)

alpha = (5.8646023550014274e-39+4.172684608813957e-47j)
-4.395408890389111e-26 0.0031835816999028067
